#### On teste différentes architectures de VAE. On prend des petites architectures  pour commencer, que l'on peut apprendre sur moins de données, et on augmentera la taille ensuite.

In [1]:
import torch
import torch.nn as nn 
from torch.utils.data import Dataset, DataLoader 
import os 
import numpy as np

In [16]:
test_wav = np.load("TrainValTest/testSet.npy")


## A. Première comparaison : VAE1D avec VAE2D

In [36]:
from downloader import SpectroDataset
from downloader import SpectroDataset4D
import models.simpleAE as AE
import models.vanillaVAE as VAE
import importlib as implib

implib.reload(AE)

<module 'models.simpleAE' from '/home/hippo/Documents/voiceVAE/models/simpleAE.py'>

In [37]:
validation_path = test_wav
torch.manual_seed(123)
spectros1 = SpectroDataset(validation_path)
valid_loader1 = DataLoader(spectros1,batch_size=16,shuffle=False)
spectros2 = SpectroDataset4D(validation_path)
valid_loader2 = DataLoader(spectros2,batch_size=16,shuffle=False)

#### les modèles testés doivent avoir été entrainés auparavant pour récupération des poids

### Modèle 1 : AE1D

In [38]:
model1 = AE.AE1D(32)
state_dic = torch.load("modelsParam/AE1D/ep5AE1D.pth")
model1.load_state_dict(state_dic)

<All keys matched successfully>

### Modèle 2 : AE2D

In [39]:
model2= AE.AE2D(32)
#state_dic = torch.load("modelsParam/AE2D/ep5AE2D.pth")
#model2.load_state_dict(state_dic)

# verification comparabilité des modèles 

#### même nombre de paramètres

In [29]:
#nombre de paramètres du modèle
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

In [40]:
print(count_parameters(model1))
print(count_parameters(model2))

41845
596384


#### Entrainement similaire (même nb d'epochs, batch, lr, nb de données, etc...)

## 1. Visualisation des reconstructions et comparaisons 2 à 2 de modèles 

In [41]:
batch1 = next(iter(valid_loader1))
batch2 = next(iter(valid_loader2))
recon1 = model1(batch1)
recon2 = model2(batch2)

In [ ]:
import matplotlib.pyplot as plt

#affichage des reocnstruction et des spectros originaux 
plt.rcParams['figure.figsize'] = [100, 50]
for i in range(2):
    print('SPECTRO n°',i,' :')
    print('Spectro original :')
    plt.imshow(batch1[i].detach().numpy())
    plt.show()
    print('Reconstruction 1 :')
    plt.imshow(recon1[i].detach().numpy())
    plt.show()
    print('Reconstruction 2 :')
    plt.imshow(np.transpose(recon2[i][0].detach().numpy()))
    plt.show()


## 2. Etude des espaces latents construits